In [17]:
import qiskit
import qiskit_aer
import qiskit_nature
import qiskit_nature_pyscf
import qiskit_algorithms
import qiskit_nature_pyscf
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.circuit.library import EfficientSU2
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_aer import  AerSimulator
# from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from scipy.linalg import eigh
from qiskit_nature.second_q.circuit.library import HartreeFock
from scipy.linalg import expm
from qiskit.quantum_info import Statevector
from numpy import pi
import pylatexenc
import numpy as np
import matplotlib.pyplot as plt
from qiskit_addon_sqd.counts import counts_to_arrays
from qiskit_addon_sqd.qubit import solve_qubit

In [5]:
# Check the installed Qiskit packages
! pip list | grep qiskit

qiskit                    1.4.0
qiskit-addon-sqd          0.10.0
qiskit-aer                0.17.0
qiskit-algorithms         0.3.1
qiskit-nature             0.7.2
qiskit-nature-pyscf       0.4.0


In [9]:
# Define the Li molecule
molecule = MoleculeInfo(
    symbols=["Li", "H"],
    coords=([0.0, 0.0, 0.0], [0.0, 0.0, 1.59]),  # Approximate bond distance for He2
    multiplicity=1,  # Singlet state
    charge=0
)

# Set up the PySCF driver
mdriver = PySCFDriver.from_molecule(
    molecule=molecule,
    basis="sto3g"
)


# You can now use the driver to run a calculation
electronic_structure_problem = mdriver.run()

In [10]:
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(electronic_structure_problem.second_q_ops()[0])

In [11]:
from KQSCI_module import *

In [12]:
evolved_state_file = "state_probabilities_pi.txt"
sample_space_size = 3
kqsci_size = 3
num_qubits = 12
state_probabilities = read_states(evolved_state_file)
# eigenvalues, eigenvectors = kqsci_eign_SVD(state_probabilities, num_qubits, sample_space_size, kqsci_size, qubit_hamiltonian)

In [ ]:
# 0.998447567774 + eigenvalues

array([-7.88026774, -7.12352517, -6.79921234])

In [31]:
evolved_state_file = "state_probabilities_pi.txt"
state_probabilities = read_states(evolved_state_file)
max_d = 20
max_k = 10


In [15]:
def selection_sqd(state_probabilities, sample_space_size):
    result = {}
    print("Before normalising the states:")
    for time in sorted(state_probabilities.keys()):
        # Get the state probabilities for the current time
        probabilities = state_probabilities[time]

        # Sort states by their probabilities in descending order and take the top states
        top_states = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)[:sample_space_size]

        # Store the top states and their probabilities in the result dictionary
        result[time] = top_states
    return result

In [16]:
selection_sqd(state_probabilities, sample_space_size)

Before normalising the states:


{0.0031415: [('000011000011', 1.0)],
 0.35184800000000005: [('000011000011', 0.99766),
  ('100001100001', 0.00173),
  ('100001000101', 0.00021)],
 0.7005545000000001: [('000011000011', 0.9906),
  ('100001100001', 0.00714),
  ('000101100001', 0.00062)],
 1.0492610000000002: [('000011000011', 0.98002),
  ('100001100001', 0.01512),
  ('100001000101', 0.0012)],
 1.3979675: [('000011000011', 0.96577),
  ('100001100001', 0.02503),
  ('100001000101', 0.00254)],
 1.7466740000000003: [('000011000011', 0.95161),
  ('100001100001', 0.03493),
  ('100001000101', 0.00337)],
 2.0953805: [('000011000011', 0.93775),
  ('100001100001', 0.04145),
  ('100001000101', 0.00439)],
 2.444087: [('000011000011', 0.92314),
  ('100001100001', 0.04895),
  ('100001000101', 0.00651)],
 2.7927935: [('000011000011', 0.91261),
  ('100001100001', 0.05059),
  ('100001000101', 0.00702)],
 3.1415: [('000011000011', 0.90448),
  ('100001100001', 0.04935),
  ('100010000011', 0.00837)]}

In [57]:
# with open("LiH_KQSCI_result_sqd_d_k.txt", "a") as file:
# state_probabilities = selection_sqd(state_probabilities, sample_space_size)
result  = selection_sqd(state_probabilities, sample_space_size)
states={}
for time in list(result.keys()):
    if time != 0:
        # Perform SQD analysis for the current time step
        counts = result[time]
    #     # Perform SQD analysis for the current time step
    #     chis, coeff =  counts_to_arrays(counts)
    #     eignvalues, eignvector  = solve_qubit(chis, qubit_hamiltonian)
    #     # Print the results
    #     print(f"Time: {time}, Eigenvalues: {eignvalues[0]}, GS_energy: {eignvalues[0]+0.998447567774}")
    #     file.write(f"Time:{time} , GS_energy:{eignvalues[0]+0.998447567774}\n")
    #     file.flush()
    # # Skip the first time step
    #     # Perform SQD analysis for the current time step
        for key, value in counts:
            if key in states:
                states[key] += value
            else:
                states[key] = value
        # # print(sorted(states.items(), key=lambda x: x[1], reverse=True)[:sample_space_size])
        # if key in states:
        #     states[key] += 
        # else:
        #     states[key] = [value]
        # Normalize the state probabilities
        total_probability = sum(states.values())
        for key in states:
            states[key] /= total_probability
        # print(states)

    else:
        # Skip the first time step
        continue

chis, coeff =  counts_to_arrays(states)
# eignvalues, eignvector  = solve_qubit(chis, qubit_hamiltonian)
# # Print the results
# # print(f"Time: {time}, Eigenvalues: {eignvalues[0]}, GS_energy: {eignvalues[0]+0.998447567774}")
# # file.write(f"Time:{time} , GS_energy:{eignvalues[0]+0.998447567774}\n")
# # file.flush()
# eignvalues
states

Before normalising the states:


{'000011000011': 0.9435117797297868,
 '100001100001': 0.04909952447177657,
 '100001000101': 0.0031204933117247855,
 '000101100001': 2.58226451219238e-06,
 '100010000011': 0.004265620222199572}